<a href="https://colab.research.google.com/github/Vivek-afk81/pyspark-learning-notes/blob/main/pySpark1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import os
import pyspark
import pandas as pd

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
# Change to a specific directory
os.chdir('/content/drive/My Drive/pyspark')

# Verify current directory
print(os.getcwd())

/content/drive/My Drive/pyspark


In [13]:
pd.read_excel("sample_pyspark_data.xlsx")

,id,name,age,department,salary
0,1,Alice,24,IT,50000
1,2,Bob,28,HR,45000
2,3,Charlie,32,Finance,60000
3,4,David,26,IT,52000
4,5,Eva,30,Marketing,48000


In [26]:
type(pd.read_csv("student_practice_data.csv"))

pandas.core.frame.DataFrame

##Working with Spark

1. Create a sprark session

In [14]:
from pyspark.sql import SparkSession

In [15]:
spark=SparkSession.builder.appName('Practise').getOrCreate()

In [16]:
spark

In [20]:
df_pyspark=spark.read.csv("student_practice_data.csv")
df_pyspark.show()

+----------+-----+---+--------------+-----+---------+
|       _c0|  _c1|_c2|           _c3|  _c4|      _c5|
+----------+-----+---+--------------+-----+---------+
|student_id| name|age|        course|score|     city|
|       101|Aarav| 20|            AI|   85|    Delhi|
|       102| Diya| 21|  Data Science|   90|   Mumbai|
|       103|Karan| 19|Cyber Security|   78|     Pune|
|       104|Meera| 22|            AI|   88|Bangalore|
|       105|Rohit| 20|       Web Dev|   82|  Chennai|
+----------+-----+---+--------------+-----+---------+



### Reading the data

In [30]:
# to remove these headers(_c0,_c1,....)
df_pyspark=spark.read.option('header','true').csv('student_practice_data.csv')

In [27]:
type(df_pyspark)

pyspark.sql.classic.dataframe.DataFrame

In [31]:
df_pyspark.head(3)

[Row(student_id='101', name='Aarav', age='20', course='AI', score='85', city='Delhi'),
 Row(student_id='102', name='Diya', age='21', course='Data Science', score='90', city='Mumbai'),
 Row(student_id='103', name='Karan', age='19', course='Cyber Security', score='78', city='Pune')]

In [32]:
#basically this works as info
df_pyspark.printSchema()

root
 |-- student_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- course: string (nullable = true)
 |-- score: string (nullable = true)
 |-- city: string (nullable = true)



In [35]:
# df_pyspark=spark.read.option('header','true').csv('student_practice_data.csv',inferSchema=True)
#or we can use
df_pyspark=spark.read.csv('student_practice_data.csv',header=True,inferSchema=True)
df_pyspark.show()


+----------+-----+---+--------------+-----+---------+
|student_id| name|age|        course|score|     city|
+----------+-----+---+--------------+-----+---------+
|       101|Aarav| 20|            AI|   85|    Delhi|
|       102| Diya| 21|  Data Science|   90|   Mumbai|
|       103|Karan| 19|Cyber Security|   78|     Pune|
|       104|Meera| 22|            AI|   88|Bangalore|
|       105|Rohit| 20|       Web Dev|   82|  Chennai|
+----------+-----+---+--------------+-----+---------+



### Checking the Datatypes of the Column(Schema)

In [36]:
df_pyspark.printSchema()

root
 |-- student_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- course: string (nullable = true)
 |-- score: integer (nullable = true)
 |-- city: string (nullable = true)



###Selecting Columns and Indexing

In [37]:
df_pyspark.columns

['student_id', 'name', 'age', 'course', 'score', 'city']

In [38]:
df_pyspark.show()

+----------+-----+---+--------------+-----+---------+
|student_id| name|age|        course|score|     city|
+----------+-----+---+--------------+-----+---------+
|       101|Aarav| 20|            AI|   85|    Delhi|
|       102| Diya| 21|  Data Science|   90|   Mumbai|
|       103|Karan| 19|Cyber Security|   78|     Pune|
|       104|Meera| 22|            AI|   88|Bangalore|
|       105|Rohit| 20|       Web Dev|   82|  Chennai|
+----------+-----+---+--------------+-----+---------+



In [41]:
df_pyspark.select('name').show()

#selecting multiple columns
df_pyspark.select('student_id','name','city').show()

+-----+
| name|
+-----+
|Aarav|
| Diya|
|Karan|
|Meera|
|Rohit|
+-----+

+----------+-----+---------+
|student_id| name|     city|
+----------+-----+---------+
|       101|Aarav|    Delhi|
|       102| Diya|   Mumbai|
|       103|Karan|     Pune|
|       104|Meera|Bangalore|
|       105|Rohit|  Chennai|
+----------+-----+---------+



In [40]:
#Notice that the .select is returning a dataframe object
type(df_pyspark.select('name'))

pyspark.sql.classic.dataframe.DataFrame

In [42]:
#Checking the datatypes
df_pyspark.dtypes

[('student_id', 'int'),
 ('name', 'string'),
 ('age', 'int'),
 ('course', 'string'),
 ('score', 'int'),
 ('city', 'string')]

In [44]:
df_pyspark.describe().show()

+-------+------------------+-----+-----------------+-------+----------------+---------+
|summary|        student_id| name|              age| course|           score|     city|
+-------+------------------+-----+-----------------+-------+----------------+---------+
|  count|                 5|    5|                5|      5|               5|        5|
|   mean|             103.0| NULL|             20.4|   NULL|            84.6|     NULL|
| stddev|1.5811388300841898| NULL|1.140175425099138|   NULL|4.77493455452533|     NULL|
|    min|               101|Aarav|               19|     AI|              78|Bangalore|
|    max|               105|Rohit|               22|Web Dev|              90|     Pune|
+-------+------------------+-----+-----------------+-------+----------------+---------+



### Adding And Dropping Columns

In [48]:
'''Returns a new DataFrame by adding a column or replacing the
existing column that has the same name.

The column expression must be an expression over this DataFrame; attempting to add
a column from some other DataFrame will raise an error.'''
from pyspark.sql.functions import rand, floor

df_pyspark = df_pyspark.withColumn(
    "Score2",
    df_pyspark["score"] + floor(rand() * 6)
)
df_pyspark.show()

+----------+-----+---+--------------+-----+---------+------+
|student_id| name|age|        course|score|     city|Score2|
+----------+-----+---+--------------+-----+---------+------+
|       101|Aarav| 20|            AI|   85|    Delhi|    85|
|       102| Diya| 21|  Data Science|   90|   Mumbai|    93|
|       103|Karan| 19|Cyber Security|   78|     Pune|    81|
|       104|Meera| 22|            AI|   88|Bangalore|    93|
|       105|Rohit| 20|       Web Dev|   82|  Chennai|    87|
+----------+-----+---+--------------+-----+---------+------+



In [51]:
# Dropping a columns
df_pyspark=df_pyspark.drop('Score2')
df_pyspark.show()

+----------+-----+---+--------------+-----+---------+
|student_id| name|age|        course|score|     city|
+----------+-----+---+--------------+-----+---------+
|       101|Aarav| 20|            AI|   85|    Delhi|
|       102| Diya| 21|  Data Science|   90|   Mumbai|
|       103|Karan| 19|Cyber Security|   78|     Pune|
|       104|Meera| 22|            AI|   88|Bangalore|
|       105|Rohit| 20|       Web Dev|   82|  Chennai|
+----------+-----+---+--------------+-----+---------+



In [52]:
# Rename a columns

df_pyspark.withColumnRenamed('name','Name').show()

+----------+-----+---+--------------+-----+---------+
|student_id| Name|age|        course|score|     city|
+----------+-----+---+--------------+-----+---------+
|       101|Aarav| 20|            AI|   85|    Delhi|
|       102| Diya| 21|  Data Science|   90|   Mumbai|
|       103|Karan| 19|Cyber Security|   78|     Pune|
|       104|Meera| 22|            AI|   88|Bangalore|
|       105|Rohit| 20|       Web Dev|   82|  Chennai|
+----------+-----+---+--------------+-----+---------+



###Handling missing values

1. Dropping Columns
2. Dropping Rows
3. Various Parameter in Dropping Functionalities

4. Handeling Missing values by Mean,Median and Mode

In [56]:
DF=spark.read.csv("missing_values_practice.csv",header=True,inferSchema=True)
DF.show()

+-----------+-----+----+----------+----------------+------+
|employee_id| name| age|department|experience_years|rating|
+-----------+-----+----+----------+----------------+------+
|        201| Amit|25.0|        IT|             2.0|   4.5|
|        202| Neha|NULL|        HR|             3.0|  NULL|
|        203| Ravi|30.0|      NULL|            NULL|   3.8|
|        204|Priya|28.0|   Finance|             5.0|  NULL|
|        205| NULL|35.0|        IT|             7.0|   4.2|
|        206|Suman|NULL|        HR|            NULL|   3.9|
+-----------+-----+----+----------+----------------+------+



In [68]:
#Let's add a column Salary also
from pyspark.sql.functions import when, col,rand


DF = DF.withColumn(
    "Salary",
    floor(rand() * 50000 + 30000)
)

#now introducing null values
DF = DF.withColumn(
    "Salary",
    when(rand() < 0.2, None)
    .otherwise(col("Salary"))
)

DF.show()

+-----------+-----+----+----------+----------------+------+------+
|employee_id| name| age|department|experience_years|rating|Salary|
+-----------+-----+----+----------+----------------+------+------+
|        201| Amit|25.0|        IT|             2.0|   4.5| 44999|
|        202| Neha|NULL|        HR|             3.0|  NULL|  NULL|
|        203| Ravi|30.0|      NULL|            NULL|   3.8| 58296|
|        204|Priya|28.0|   Finance|             5.0|  NULL| 78473|
|        205| NULL|35.0|        IT|             7.0|   4.2|  NULL|
|        206|Suman|NULL|        HR|            NULL|   3.9| 76467|
+-----------+-----+----+----------+----------------+------+------+



In [70]:
#This will remove all those rows which ahve null values present in it
DF.na.drop().show()

+-----------+----+----+----------+----------------+------+------+
|employee_id|name| age|department|experience_years|rating|Salary|
+-----------+----+----+----------+----------------+------+------+
|        201|Amit|25.0|        IT|             2.0|   4.5| 44999|
+-----------+----+----+----------+----------------+------+------+



In [73]:
# any=how
DF.na.drop(how='all').show() # it will drop those rows which have all null values in this case there are'nt any
DF.na.drop(how='any').show()

+-----------+-----+----+----------+----------------+------+------+
|employee_id| name| age|department|experience_years|rating|Salary|
+-----------+-----+----+----------+----------------+------+------+
|        201| Amit|25.0|        IT|             2.0|   4.5| 44999|
|        202| Neha|NULL|        HR|             3.0|  NULL|  NULL|
|        203| Ravi|30.0|      NULL|            NULL|   3.8| 58296|
|        204|Priya|28.0|   Finance|             5.0|  NULL| 78473|
|        205| NULL|35.0|        IT|             7.0|   4.2|  NULL|
|        206|Suman|NULL|        HR|            NULL|   3.9| 76467|
+-----------+-----+----+----------+----------------+------+------+

+-----------+----+----+----------+----------------+------+------+
|employee_id|name| age|department|experience_years|rating|Salary|
+-----------+----+----+----------+----------------+------+------+
|        201|Amit|25.0|        IT|             2.0|   4.5| 44999|
+-----------+----+----+----------+----------------+------+------+

In [78]:
## Threshold
DF.na.drop(how="any",thresh=5).show() #it says atleast 5 non null values must be there, in this case only 2nd row has 4 non null values hence it got dropped

+-----------+-----+----+----------+----------------+------+------+
|employee_id| name| age|department|experience_years|rating|Salary|
+-----------+-----+----+----------+----------------+------+------+
|        201| Amit|25.0|        IT|             2.0|   4.5| 44999|
|        203| Ravi|30.0|      NULL|            NULL|   3.8| 58296|
|        204|Priya|28.0|   Finance|             5.0|  NULL| 78473|
|        205| NULL|35.0|        IT|             7.0|   4.2|  NULL|
|        206|Suman|NULL|        HR|            NULL|   3.9| 76467|
+-----------+-----+----+----------+----------------+------+------+



In [80]:
#Subset when you want to drop nan values with respect to a specific columns
DF.na.drop(how='any',subset=['experience_years']).show()

+-----------+-----+----+----------+----------------+------+------+
|employee_id| name| age|department|experience_years|rating|Salary|
+-----------+-----+----+----------+----------------+------+------+
|        201| Amit|25.0|        IT|             2.0|   4.5| 44999|
|        202| Neha|NULL|        HR|             3.0|  NULL|  NULL|
|        204|Priya|28.0|   Finance|             5.0|  NULL| 78473|
|        205| NULL|35.0|        IT|             7.0|   4.2|  NULL|
+-----------+-----+----+----------+----------------+------+------+



In [84]:
# Filling the Missing Values
DF.na.fill('Missing_values').show() #Spark interprets this as:“Fill STRING columns only with 'Missing_values'”
DF.na.fill('Missing_values',['experience_years','age']).show()#Spark will NOT insert a string into numeric columns

+-----------+--------------+----+--------------+----------------+------+------+
|employee_id|          name| age|    department|experience_years|rating|Salary|
+-----------+--------------+----+--------------+----------------+------+------+
|        201|          Amit|25.0|            IT|             2.0|   4.5| 44999|
|        202|          Neha|NULL|            HR|             3.0|  NULL|  NULL|
|        203|          Ravi|30.0|Missing_values|            NULL|   3.8| 58296|
|        204|         Priya|28.0|       Finance|             5.0|  NULL| 78473|
|        205|Missing_values|35.0|            IT|             7.0|   4.2|  NULL|
|        206|         Suman|NULL|            HR|            NULL|   3.9| 76467|
+-----------+--------------+----+--------------+----------------+------+------+

+-----------+-----+----+----------+----------------+------+------+
|employee_id| name| age|department|experience_years|rating|Salary|
+-----------+-----+----+----------+----------------+------+------